In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Projects/Nuclear Physics GPT/Data/nuclear_physics_clean_df.csv', index_col=0).reset_index(drop=True)

df = df.dropna()
df.tail()

,Protons,Neutrons,Atomic Weight,Mass Excess,Binding Energy,Half-life,Half-life Magnitude,Half-life Time
3176,116,175,291.200011,186.309,7.119,0.006300,-3.0,2.266447
3177,116,176,292.199786,186.100,7.123,0.018000,-2.0,2.418424
3178,117,174,291.206564,192.413,7.096,0.009955,-3.0,2.332675
3179,117,175,292.207549,193.330,7.096,0.050000,-2.0,2.566323
3182,118,175,293.214670,199.964,7.074,0.005000,-3.0,2.232990


In [3]:
def calculate_atomic_weight(proton, neutron):
  PROTON_MASS = 1.007276 # in atomic mass units
  NEUTRON_MASS = 1.008664 # in atomic mass units

  atomic_weight = proton * PROTON_MASS + neutron * NEUTRON_MASS
  return atomic_weight

def create_border_elements(df):
  # Identify the "top" border elements
  unique_protons = df['Protons'].unique()
  top_border_neutrons = {proton: df[df['Protons'] == proton]['Neutrons'].max() + 1 for proton in unique_protons}
  # Identify the "bottom" border elements
  bottom_border_neutrons = {proton: df[df['Protons'] == proton]['Neutrons'].min() - 1 for proton in unique_protons if df[df['Protons'] == proton]['Neutrons'].min() - 1 >= 0}

  border_elements = []
  for proton, neutron in list(top_border_neutrons.items()) + list(bottom_border_neutrons.items()):
    atomic_weight = calculate_atomic_weight(proton, neutron)
    border_elements.append([proton, neutron, atomic_weight, 0, 0, 0, 0, 0, 1])

  border_df = pd.DataFrame(border_elements, columns=df.columns)
  return pd.concat([df, border_df]).reset_index(drop=True)


#def create_super_heavy_elements(df):
def create_super_heavy_elements(df):
    # Define the range for super-heavy protons and neutrons
    super_heavy_protons = range(121, 200)  # 120 to 160
    super_heavy_neutrons = range(165, 301)  # 150 to 300

    super_heavy_elements = []

    # Generate combinations of super-heavy protons and neutrons
    for proton in super_heavy_protons:
        for neutron in super_heavy_neutrons:
            atomic_weight = calculate_atomic_weight(proton, neutron)
            # Append the data as a list; here I'm assuming that the remaining columns should be filled with zeros, similar to your border elements
            super_heavy_elements.append([proton, neutron, atomic_weight, 0, 0, 0, 0, 0, 0])

    # Create a new dataframe for the super-heavy elements
    super_heavy_df = pd.DataFrame(super_heavy_elements, columns=df.columns)

    # Concatenate the original dataframe with the super-heavy elements dataframe
    return pd.concat([df, super_heavy_df]).reset_index(drop=True)

def create_end_elements(df):
    # Define the range for super-heavy protons and neutrons
    super_heavy_protons = range(119, 121)  # 120 to 160
    super_heavy_neutrons = range(173, 176)  # 150 to 300

    super_heavy_elements = []

    # Generate combinations of super-heavy protons and neutrons
    for proton in super_heavy_protons:
        for neutron in super_heavy_neutrons:
            atomic_weight = calculate_atomic_weight(proton, neutron)
            # Append the data as a list; here I'm assuming that the remaining columns should be filled with zeros, similar to your border elements
            super_heavy_elements.append([proton, neutron, atomic_weight, 0, 0, 0, 0, 0, 0])

    # Create a new dataframe for the super-heavy elements
    super_heavy_df = pd.DataFrame(super_heavy_elements, columns=df.columns)

    # Concatenate the original dataframe with the super-heavy elements dataframe
    return pd.concat([df, super_heavy_df]).reset_index(drop=True)

In [4]:
df['Artificial'] = 0
df = create_border_elements(df)
df = create_border_elements(df)
df = create_super_heavy_elements(df)
df = create_end_elements(df)

df['N-Z'] = df['Neutrons'] - df['Protons']
df['Z-N'] = df['Protons'] - df['Neutrons']

df['Abs(Z-N)'] = abs(df['Protons'] - df['Neutrons'])

df['N/Z'] = np.where(df['Protons'] > 0, df['Neutrons'] / df['Protons'], 0)
df['Z/N'] = np.where(df['Neutrons'] > 0, df['Protons'] / df['Neutrons'], 0)

df['Z/Atomic Weight'] = df['Protons'] / df['Atomic Weight']
df['N/Atomic Weight'] = df['Neutrons'] / df['Atomic Weight']

df['Abs(Z-N)/Atomic Weight'] = df['Abs(Z-N)'] / df['Atomic Weight']

df

,Protons,Neutrons,Atomic Weight,Mass Excess,Binding Energy,Half-life,Half-life Magnitude,Half-life Time,Artificial,N-Z,Z-N,Abs(Z-N),N/Z,Z/N,Z/Atomic Weight,N/Atomic Weight,Abs(Z-N)/Atomic Weight
0,1,0,1.007825,7.288970,0.000000,2.486929e+62,62.0,8.000000e+00,0,-1,1,1,0.000000,0.000000,0.992236,0.000000,0.992236
1,1,1,2.014102,13.135722,1.112283,2.486929e+62,62.0,8.000000e+00,0,0,0,0,1.000000,1.000000,0.496499,0.496499,0.000000
2,1,2,3.016049,14.949806,2.827266,3.885728e+08,8.0,7.019868e+00,0,1,-1,1,2.000000,0.500000,0.331560,0.663119,0.331560
3,1,3,4.027806,25.901518,1.400351,9.917391e-23,-23.0,9.917391e-17,0,2,-2,2,3.000000,0.333333,0.248274,0.744822,0.496548
4,1,4,5.035311,32.892440,1.336360,8.003509e-23,-23.0,8.003509e-17,0,3,-3,3,4.000000,0.250000,0.198597,0.794390,0.595792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14362,119,174,295.373380,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,55,-55,55,1.462185,0.683908,0.402880,0.589085,0.186205
14363,119,175,296.382044,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,56,-56,56,1.470588,0.680000,0.401509,0.590454,0.188945
14364,120,173,295.371992,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,53,-53,53,1.441667,0.693642,0.406267,0.585702,0.179435
14365,120,174,296.380656,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,54,-54,54,1.450000,0.689655,0.404885,0.587083,0.182198


In [5]:
def add_transformed_columns(df):
    new_cols = pd.DataFrame()
    for col in df.columns:
        if 'Half-life' not in col and 'Artifical' not in col:
            # Square root transform
            new_cols[f'{col}_sqrt'] = df[col].apply(lambda x: np.sqrt(x) if x > 0 else 0)
            # Log transform
            new_cols[f'{col}_log'] = df[col].apply(lambda x: np.log1p(x) if x > 0 else 0) #log(1+x)
            # Exponential transform
            new_cols[f'{col}_exp'] = df[col].apply(lambda x: np.exp(x) if x > 0 else 0)
            # Inverse (reciprocal) transform
            new_cols[f'{col}_reciprocal'] = df[col].apply(lambda x: 1/x if x!=0 else 0)
            # Power transform
            new_cols[f'{col}_power'] = df[col].apply(lambda x: x**2 if x > 0 else 0)
            # Hyperbolic Tangent transform
            new_cols[f'{col}_tanh'] = df[col].apply(np.tanh)
    return pd.concat([df, new_cols], axis=1)

# Applying the transformation function
df = add_transformed_columns(df)
df.head()

magic_protons = [2, 8, 20, 28, 50, 82, 126]

df['Is Magic Number'] = df['Protons'].isin(magic_protons)
df['Magic Number'] = df['Is Magic Number']*df['Protons']
df['Closest Magic Number'] = df['Protons'].apply(lambda x: min(magic_protons, key=lambda y: abs(x - y)))
df['Distance To Magic Number_abs'] = abs(df['Closest Magic Number'] - df['Protons'])
df['Distance To Magic Number'] = df['Protons'] - df['Closest Magic Number']

df


,Protons,Neutrons,Atomic Weight,Mass Excess,Binding Energy,Half-life,Half-life Magnitude,Half-life Time,Artificial,N-Z,...,Abs(Z-N)/Atomic Weight_log,Abs(Z-N)/Atomic Weight_exp,Abs(Z-N)/Atomic Weight_reciprocal,Abs(Z-N)/Atomic Weight_power,Abs(Z-N)/Atomic Weight_tanh,Is Magic Number,Magic Number,Closest Magic Number,Distance To Magic Number_abs,Distance To Magic Number
0,1,0,1.007825,7.288970,0.000000,2.486929e+62,62.0,8.000000e+00,0,-1,...,0.689257,2.697258,1.007825,0.984532,0.758314,False,0,2,1,-1
1,1,1,2.014102,13.135722,1.112283,2.486929e+62,62.0,8.000000e+00,0,0,...,0.000000,0.000000,0.000000,0.000000,0.000000,False,0,2,1,-1
2,1,2,3.016049,14.949806,2.827266,3.885728e+08,8.0,7.019868e+00,0,1,...,0.286351,1.393139,3.016049,0.109932,0.319921,False,0,2,1,-1
3,1,3,4.027806,25.901518,1.400351,9.917391e-23,-23.0,9.917391e-17,0,2,...,0.403161,1.643040,2.013903,0.246560,0.459398,False,0,2,1,-1
4,1,4,5.035311,32.892440,1.336360,8.003509e-23,-23.0,8.003509e-17,0,3,...,0.467370,1.814468,1.678437,0.354969,0.534049,False,0,2,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14362,119,174,295.373380,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,55,...,0.170759,1.204669,5.370425,0.034672,0.184082,False,0,126,7,-7
14363,119,175,296.382044,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,56,...,0.173067,1.207975,5.292536,0.035700,0.186729,False,0,126,7,-7
14364,120,173,295.371992,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,53,...,0.165035,1.196541,5.573056,0.032197,0.177533,False,0,126,6,-6
14365,120,174,296.380656,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,54,...,0.167376,1.199852,5.488531,0.033196,0.180208,False,0,126,6,-6


In [6]:
# Quantum number mappings for each type of atomic orbital
orbital_type_mappings = {'s': 0, 'p': 1, 'd': 2, 'f': 3, 'g': 4, 'h': 5}

# Maximum electron capacities for each type of atomic orbital
max_electron_capacity = {'s': 2, 'p': 6, 'd': 10, 'f': 14, 'g': 18, 'h': 22}

# Defined energy levels, including possible orbitals for superheavy elements
# These represent the maximum number of electrons that can be held at each energy level.
energy_levels = {
    '1s': 2, '2s': 2, '2p': 6, '3s': 2, '3p': 6, '4s': 2,
    '3d': 10, '4p': 6, '5s': 2, '4d': 10, '5p': 6, '6s': 2,
    '4f': 14, '5d': 10, '6p': 6, '7s': 2, '5f': 14, '6d': 10,
    '7p': 6, '8s': 2, '5g': 18, '8p': 6, '6f': 14, '5h': 22,
    '9s': 2, '6g': 24, '9p': 6, '7f': 14, '6h': 26, '10s': 2
}

def get_experimental_electron_config(protons):
    """
    Given a number of protons, return the electron configuration of the atom.
    """
    electron_config = {}
    for level in energy_levels:
        if protons - energy_levels[level] >= 0:
            electron_config[level] = energy_levels[level]
            protons -= energy_levels[level]
        else:
            electron_config[level] = protons
            break


    if electron_config[list(electron_config.keys())[-1]]==0:
      electron_config.popitem()
    return electron_config


def is_last_shell_filled(protons):
    """
    Given a number of protons, return whether the last energy level is fully filled.
    """
    electron_config = get_experimental_electron_config(protons)
    last_level, last_level_electrons = list(electron_config.items())[-1]
    last_level_type = last_level[-1]
    return int(last_level_electrons == max_electron_capacity[last_level_type])

def last_shell_fill_ratio(protons):
    """
    Given a number of protons, return the fill ratio of the last energy level.
    """
    electron_config = get_experimental_electron_config(protons)
    last_level, last_level_electrons = list(electron_config.items())[-1]
    last_level_type = last_level[-1]
    return last_level_electrons / max_electron_capacity[last_level_type]

def get_last_level_encoded(protons):
    """
    Given a number of protons, return an encoded value representing the last energy level.
    """
    electron_config = get_experimental_electron_config(protons)
    last_level = list(electron_config.keys())[-1]
    n = int(last_level[:-1])
    l = last_level[-1]
    return 10 * n + orbital_type_mappings[l]

def get_last_level_encoded_simple(protons):

    # Get the electron configuration
    electron_config = get_experimental_electron_config(protons)

    # Define the energy levels in the order of filling
    energy_levels_ordered = ['1s', '2s', '2p', '3s', '3p', '4s', '3d', '4p', '5s', '4d', '5p', '6s', '4f', '5d', '6p',
                             '7s', '5f', '6d', '7p', '8s', '5g', '8p', '6f', '5h', '9s', '6g', '9p', '7f', '6h', '10s']

    # Define a mapping from energy level to its order of filling
    energy_level_encoding_simple = {level: i for i, level in enumerate(energy_levels_ordered)}

    # Get the last energy level
    last_level = list(electron_config.keys())[-1]

    # Return the simple encoded value
    return energy_level_encoding_simple[last_level]


def get_last_principal_quantum_number(protons):
    # Get the electron configuration
    electron_config = get_experimental_electron_config(protons)

    # Extract the last principal quantum number from the keys of the electron configuration
    return int(list(electron_config.keys())[-1][:-1])

def get_last_azimuthal_quantum_number_encoded(protons):
    # Get the electron configuration
    electron_config = get_experimental_electron_config(protons)

    # Extract the last azimuthal quantum number from the keys of the electron configuration
    last_azimuthal_quantum_number = list(electron_config.keys())[-1][-1]

    # Return the encoded value
    return orbital_type_mappings[last_azimuthal_quantum_number]

In [7]:
# Create a new DataFrame containing the new columns
new_columns = pd.DataFrame({
    #'Electron_Config': df['Protons'].apply(get_experimental_electron_config),
    'Last Shell Filled': df['Protons'].apply(is_last_shell_filled),
    'Last Shell Fill Ratio': df['Protons'].apply(last_shell_fill_ratio),
    'Last Orbital Label Encoded (Complex)': df['Protons'].apply(get_last_level_encoded),
    'Last Orbital Label Encoded (Simple)': df['Protons'].apply(get_last_level_encoded_simple) + 1,
    'Last Principal Quantum Number': df['Protons'].apply(get_last_principal_quantum_number),
    'Last Azimuthal Quantum Number Encoded': df['Protons'].apply(get_last_azimuthal_quantum_number_encoded)
})

# Concatenate the original DataFrame with the new columns
df = pd.concat([df, new_columns], axis=1)

df['Even Z'] = df['Protons'] % 2
df['Even N'] = df['Neutrons'] % 2
df['Even N-Z'] = df['Even Z'] + 2*df['Even N']

df

<ipython-input-7-69a4b83d873e>:15: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Even Z'] = df['Protons'] % 2
<ipython-input-7-69a4b83d873e>:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Even N'] = df['Neutrons'] % 2
<ipython-input-7-69a4b83d873e>:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.c

,Protons,Neutrons,Atomic Weight,Mass Excess,Binding Energy,Half-life,Half-life Magnitude,Half-life Time,Artificial,N-Z,...,Distance To Magic Number,Last Shell Filled,Last Shell Fill Ratio,Last Orbital Label Encoded (Complex),Last Orbital Label Encoded (Simple),Last Principal Quantum Number,Last Azimuthal Quantum Number Encoded,Even Z,Even N,Even N-Z
0,1,0,1.007825,7.288970,0.000000,2.486929e+62,62.0,8.000000e+00,0,-1,...,-1,0,0.5,10,1,1,0,1,0,1
1,1,1,2.014102,13.135722,1.112283,2.486929e+62,62.0,8.000000e+00,0,0,...,-1,0,0.5,10,1,1,0,1,1,3
2,1,2,3.016049,14.949806,2.827266,3.885728e+08,8.0,7.019868e+00,0,1,...,-1,0,0.5,10,1,1,0,1,0,1
3,1,3,4.027806,25.901518,1.400351,9.917391e-23,-23.0,9.917391e-17,0,2,...,-1,0,0.5,10,1,1,0,1,1,3
4,1,4,5.035311,32.892440,1.336360,8.003509e-23,-23.0,8.003509e-17,0,3,...,-1,0,0.5,10,1,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14362,119,174,295.373380,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,55,...,-7,0,0.5,80,20,8,0,1,0,1
14363,119,175,296.382044,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,56,...,-7,0,0.5,80,20,8,0,1,1,3
14364,120,173,295.371992,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,53,...,-6,1,1.0,80,20,8,0,0,1,2
14365,120,174,296.380656,0.000000,0.000000,0.000000e+00,0.0,0.000000e+00,0,54,...,-6,1,1.0,80,20,8,0,0,0,0


In [8]:
df = df.dropna()
df = df.replace([np.inf, -np.inf], np.nan).dropna()

transformation_features = ['Protons', 'Protons_log', 'Protons_reciprocal',
                           'Neutrons', 'Neutrons_log', 'Neutrons_reciprocal', 'Neutrons_power',
                           'Atomic Weight', 'Atomic Weight_log', 'Atomic Weight_reciprocal', 'Atomic Weight_power']

special_features = ['Abs(Z-N)/Atomic Weight', 'Abs(Z-N)/Atomic Weight_reciprocal',
                            'Z-N', 'Z-N_log', 'Z-N_reciprocal',
                            'Z/N', 'Z/N_power']

even_features = ['Even Z', 'Even N', 'Even N-Z']
chem_features = ['Last Shell Filled', 'Last Shell Fill Ratio', 'Last Orbital Label Encoded (Complex)', 'Last Orbital Label Encoded (Simple)',
                     'Last Principal Quantum Number', 'Last Azimuthal Quantum Number Encoded']
magic_features = ['Is Magic Number', 'Magic Number', 'Closest Magic Number', 'Distance To Magic Number', 'Distance To Magic Number_abs']


target_features = ['Half-life', 'Half-life Magnitude', 'Half-life Time', 'Artificial']

df.loc[:, transformation_features + special_features + even_features + chem_features + magic_features + target_features]

df = df.loc[:, transformation_features + special_features + even_features + chem_features + magic_features + target_features]




In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Load the dataset
df = df.sort_values('Neutrons')

target = 'Half-life Time'
targets = ['Half-life', 'Half-life Magnitude', 'Half-life Time']
targets.remove(target)

# Drop the unnecessary columns
artifical = df['Artificial'] > 0
df = df.drop(columns=targets+['Artificial'])

df = df.drop(columns=[column for column in df.columns if 'Binding' in column])
df = df.drop(columns=[column for column in df.columns if 'Mass' in column])

# Organizing Features
base_features = ['Protons', 'Neutrons', 'Atomic Weight']
transformation_features = ['Protons_log', 'Protons_reciprocal', 'Neutrons_log', 'Neutrons_reciprocal', 'Neutrons_power', 'Atomic Weight_log', 'Atomic Weight_reciprocal', 'Atomic Weight_power']
special_features = ['Abs(Z-N)/Atomic Weight', 'Abs(Z-N)/Atomic Weight_reciprocal', 'Z-N', 'Z-N_log', 'Z-N_reciprocal', 'Z/N', 'Z/N_power',
                    'Even Z', 'Even N', 'Even N-Z',
                    'Last Shell Filled', 'Last Shell Fill Ratio', 'Last Orbital Label Encoded (Complex)', 'Last Orbital Label Encoded (Simple)', 'Last Principal Quantum Number', 'Last Azimuthal Quantum Number Encoded',
                    'Is Magic Number', 'Magic Number', 'Closest Magic Number', 'Distance To Magic Number', 'Distance To Magic Number_abs']


df

,Protons,Protons_log,Protons_reciprocal,Neutrons,Neutrons_log,Neutrons_reciprocal,Neutrons_power,Atomic Weight,Atomic Weight_log,Atomic Weight_reciprocal,...,Last Orbital Label Encoded (Complex),Last Orbital Label Encoded (Simple),Last Principal Quantum Number,Last Azimuthal Quantum Number Encoded,Is Magic Number,Magic Number,Closest Magic Number,Distance To Magic Number,Distance To Magic Number_abs,Half-life Time
0,1,0.693147,1.000000,0,0.00000,0.000000,0,1.007825,0.697052,0.992236,...,10,1,1,0,False,0,2,-1,1,8.0
3268,3,1.386294,0.333333,0,0.00000,0.000000,0,3.021828,1.391737,0.330926,...,20,2,2,0,False,0,2,1,1,0.0
3502,4,1.609438,0.250000,0,0.00000,0.000000,0,4.029104,1.615242,0.248194,...,20,2,2,0,False,0,2,2,2,0.0
3503,5,1.791759,0.200000,0,0.00000,0.000000,0,5.036380,1.797804,0.198555,...,21,3,2,1,False,0,2,3,3,0.0
3504,6,1.945910,0.166667,0,0.00000,0.000000,0,6.043656,1.952127,0.165463,...,21,3,2,1,False,0,8,-2,2,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8240,154,5.043425,0.006494,300,5.70711,0.003333,90000,457.719704,6.128439,0.002185,...,63,23,6,3,False,0,126,28,28,0.0
10008,167,5.123964,0.005988,300,5.70711,0.003333,90000,470.814292,6.156585,0.002124,...,55,24,5,5,False,0,126,41,41,0.0
6608,142,4.962845,0.007042,300,5.70711,0.003333,90000,445.632392,6.101736,0.002244,...,81,22,8,1,False,0,126,16,16,0.0
10688,172,5.153292,0.005814,300,5.70711,0.003333,90000,475.850672,6.167203,0.002101,...,55,24,5,5,False,0,126,46,46,0.0


In [10]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam, SGD
from keras.regularizers import l2
from keras.initializers import he_normal, glorot_normal
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import random

def build_model(hyperparameters):
    model = Sequential()
    input_shape = hyperparameters['input_shape']

    # Initial layer with L2 regularization and weight initialization
    model.add(Dense(hyperparameters['layer_sizes'][0], input_shape=(input_shape,), activation=hyperparameters['activations'][0], kernel_initializer=hyperparameters['weight_initializer']))

    # Batch normalization after the first layer
    if hyperparameters['batch_normalization']:
        model.add(BatchNormalization())

    # Hidden layers with Dropout
    for i in range(1, hyperparameters['n_layers']):
        model.add(Dense(hyperparameters['layer_sizes'][i], activation=hyperparameters['activations'][i], kernel_initializer=hyperparameters['weight_initializer']))
        model.add(Dropout(hyperparameters['dropout_rate']))

    # Output layer (linear activation for regression)
    model.add(Dense(1, activation='linear', kernel_initializer=hyperparameters['weight_initializer']))

    # Optimizer with learning rate and momentum (if SGD)
    if hyperparameters['optimizer'] == 'Adam':
        optimizer = Adam(learning_rate=hyperparameters['learning_rate'])
    else:
        optimizer = SGD(learning_rate=hyperparameters['learning_rate'], momentum=hyperparameters['momentum'])

    # Compile the model
    model.compile(optimizer=optimizer, loss=hyperparameters['loss'])

    return model
# Assuming df, base_features, transformation_features, special_features, and target are defined
X = df[base_features + transformation_features + special_features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.99, random_state=42)

### FINAL TEST ###
cut_off = int(.7*len(df))
X_train, y_train = X[:cut_off], y[:cut_off]
X_test, y_test = X[cut_off:], y[cut_off:]

# Normalize the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

best_hyperparameters = {'n_layers': 2,
 'layer_sizes': [X_train.shape[1], X_train.shape[1]*2],
 'activations': ['sigmoid', 'relu'],
 'optimizer': 'SGD',
 'loss': 'mean_squared_error',
 'dropout_rate': 0.3,
 'learning_rate': 0.01,
 'momentum': 0.9,
 'epochs': 50,
 'batch_size': 32,
 'batch_normalization': False,
 'weight_initializer': 'he_normal',
 'input_shape': X_train.shape[1]}

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score, median_absolute_error
from sklearn.neural_network import MLPRegressor


temp_df = df.sort_values('Protons', ascending=True)
X = temp_df[base_features + transformation_features + special_features]
y = temp_df[target]

### FINAL TEST ###
cut = .2522
cut_off = int(cut*len(df))
X_train, y_train = X[:cut_off], y[:cut_off]
X_test, y_test = X[cut_off:], y[cut_off:]

# Normalize the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = build_model(best_hyperparameters)
history = model.fit(X_train, y_train, epochs=100000, batch_size=best_hyperparameters['batch_size'], verbose=1) # Change to 10000

X_full = scaler.transform(X)
y = df[target]


y_pred_full = model.predict(X_full)
y_pred_full = [value if value > 0 else 0 for value in y_pred_full]


Streaming output truncated to the last 5000 lines.
114/114 [==============================] - 0s 3ms/step - loss: 0.4152
Epoch 17794/100000
114/114 [==============================] - 0s 3ms/step - loss: 0.3710
Epoch 17795/100000
114/114 [==============================] - 0s 2ms/step - loss: 0.3873
Epoch 17796/100000
114/114 [==============================] - 0s 3ms/step - loss: 0.3906
Epoch 17797/100000
114/114 [==============================] - 0s 4ms/step - loss: 0.4033
Epoch 17798/100000
114/114 [==============================] - 0s 4ms/step - loss: 0.3859
Epoch 17799/100000
114/114 [==============================] - 1s 4ms/step - loss: 0.3901
Epoch 17800/100000
114/114 [==============================] - 0s 3ms/step - loss: 0.3983
Epoch 17801/100000
114/114 [==============================] - 0s 3ms/step - loss: 0.3919
Epoch 17802/100000
114/114 [==============================] - 0s 3ms/step - loss: 0.3737
Epoch 17803/100000
114/114 [==============================] - 0s 3ms/step - lo

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('Model loss over epochs')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.colors as colors
import matplotlib.patches as patches

# Create a dataframe for the full dataset
df_full = pd.DataFrame(X, columns=X.columns)
df_full['Actual'] = y
df_full['Predicted'] = y_pred_full


df_full = df_full[((df_full['Predicted'] > 0) + (df_full['Protons'] < 120)) * (df_full['Protons'] <= 200)]

# Define the colormap
cmap = plt.cm.viridis  # or any other colormap you like

# Define the normalization
vmin = min(df_full['Actual'].min(), df_full['Predicted'].min())
vmax = max(df_full['Actual'].max(), df_full['Predicted'].max())
norm = colors.Normalize(vmin=vmin, vmax=vmax)
#norm = colors.LogNorm(vmin=df['Half-life Magnitude'].min()+1e-5, vmax=df['Half-life'].max())
s = 16


# Filter the dataframe to only include rows with more than 100 protons
df_filtered = df_full[df_full['Protons'] > 70]

# Create the plot
plt.figure(figsize=(10, 7))

for y in range(80, 220+1, 20):  # Here, 5 is the spacing between lines
    plt.axhline(y=y, color='grey', linewidth=0.5, alpha=0.5, zorder=0)

# Plot the predicted half-life magnitudes for filtered data
scatter = plt.scatter(df_filtered['Protons'], df_filtered['Neutrons'], c=df_filtered['Predicted'], cmap=cmap, norm=norm, alpha=1, s=s, marker='o')
plt.title('Predicted ' + 'Half Life of Super Heavy Elements')
plt.xlabel('Protons')
plt.ylabel('Neutrons')

plt.gca().add_patch(patches.Rectangle((119, 178), 15, 42, linewidth=2, edgecolor='r', facecolor='none'))
plt.text(104, 207, '      Predicted\nIsland of Stability', color='red')




# Create colorbar and get its object
cbar = plt.colorbar(scatter, label=target)

# Explicitly set ticks and tick labels
cbar.set_ticks([0, 1, 2, 3, 4, 5, 6, 7, 8])
cbar.set_ticklabels(['0', 'Microsecond', 'Millisecond', 'Second', 'Minute', 'Hour', 'Day', 'Years', '8'])

plt.show()



In [ ]:
temp_df = df.sort_values('Protons', ascending=True)
X = temp_df[base_features + transformation_features + special_features]
y = temp_df[target]

### FINAL TEST ###
cut = .2522
cut_off = int(cut*len(df))
X_train, y_train = X[:cut_off], y[:cut_off]
X_test, y_test = X[cut_off:], y[cut_off:]


X[:cut_off].tail(10)